In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import FL_utils.FL_interpretable_functions as interpret
import FL_utils.FL_RiskSLIM as slim
import FL_utils.FL_stumps as stumps
import FL_utils.FL_Corel as Corel

# restore saved variables
%store -r summary_felony2_fl_inter_model

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\FL Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['felony_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['felony_two_year'].values

In [3]:
#### CART
depth = [1,2,3]
impurity = [0.001, 0.003]
split = [2,3,4]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, split, impurity, seed=816)

### EBM
estimators = [60,80,100]
depth = [1,2]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, holdout_split, seed=816)

In [4]:
cart_summary['KY_score'], ebm_summary['KY_score']

(0.5505293446474429, 0.7049926457559672)

### Lasso Stumps

In [5]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['felony_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['felony_two_year'].values

## columns 
cols = KY_X.columns

In [6]:
c_grid={'C': [0.03, 0.04, 0.05, 0.06, 0.07]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [15]:
stumps_summary

{'best_auc': 0.6376099475946679,
 'best_params': {'C': 0.07},
 'auc_diffs': 0.03503744214084892,
 'KY_score': 0.6871795155932277}

### RiskSLIM

In [22]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['felony_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['felony_two_year'].values

## columns 
cols = KY_X.columns

In [23]:
## train on best param chosen by Lasso Stumps from above
lasso = LogisticRegression(class_weight='balanced', solver='liblinear', penalty='l1', C=0.04, random_state=816).fit(FL_X, FL_Y)
selected_features = cols[lasso.coef_[0] != 0].tolist()
len(selected_features)

14

In [24]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_FL_X.insert(0, '(Intercept)', 1)

In [25]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'felony_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000,
                                max_offset=100,
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
10/20/19 @ 12:12 PM | 193 rows in lookup table
10/20/19 @ 12:12 PM | ------------------------------------------------------------
10/20/19 @ 12:12 PM | runnning initialization procedure
10/20/19 @ 12:12 PM | ------------------------------------------------------------
10/20/19 @ 12:12 PM | CPA produced 2 cuts
10/20/19 @ 12:12 PM | running naive rounding on 4 solutions
10/20/19 @ 12:12 PM | best objective value: 0.6931
10/20/19 @ 12:12 PM | rounding produced 4 integer solutions
10/20/19 @ 12:12 PM | best objective value is 0.6931
10/20/19 @ 12:12 PM | running sequential rounding on 4 solutions
10/20/19 @ 12:12 PM | best objective value: 0.6931
10/20/19 @ 12:12 PM | sequential rounding produced 0 integer solutions
10/20/19 @ 12:12 PM | polishing 4 solutions
10/20/19 @ 12:12 PM | best objective value: 0.6931
10/20/19 @ 12:12 PM | polishing produced 4 integer solutions
10/20/19 @ 12:12 PM | initialization produced 7 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 12:12 PM | adding 179 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4518.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 441192 47233        0.4441     9        0.4518        0.4428  1240436    1.99%           rho_4 U 441192 441190     32
 451596 49297        0.4447     7        0.4518        0.4428  1263924    1.98%           rho_6 U 451596 451594     25
 461780 50964        0.4438     6        0.4518        0.4429  1290479    1.96%           rho_3 D 461780 461778     35
 471433 52106        0.4507     1        0.4518        0.4429  1311608    1.96%           rho_0 D 471433 471432     30
 481092 53607        cutoff              0.4518        0.4430  1335755    1.95%           rho_9 N 481092 481091     35
 490944 55268        0.4488     1        0.4518        0.4430  1359424    1.94%           rho_3 U 490944 490942     40
 501080 56501        cutoff              0.4518        0.4430  1383597    1.94%           rho_7 D 501080 501079     36
 510082 56914        0.4430     6        0.4518        0.4430  1408689    1.94%           rho_2 U 510082 510080     37
Elapsed time = 45.73 sec. (50790.28 ticks, tree 

 1661894 38832        0.4492     6        0.4518        0.4492  3975573    0.57%           rho_0 U 1661894 1530441     25
 1702847 30563        0.4499     9        0.4518        0.4497  4057783    0.46%           rho_7 D 1702847 1702845     33
 1743725 17972        0.4505     9        0.4518        0.4505  4136431    0.28%           rho_7 U 1743725 1743723     27
 1784557   797        cutoff              0.4518        0.4516  4206595    0.03%           rho_0 U 1784557 740468     37
Elapsed time = 135.20 sec. (168095.71 ticks, tree = 5.69 MB, solutions = 1)

Gomory fractional cuts applied:  1
User cuts applied:  167

Root node processing (before b&c):
  Real time             =    0.16 sec. (1.52 ticks)
Sequential b&c:
  Real time             =  135.13 sec. (168347.55 ticks)
                          ------------
Total (root+branch&cut) =  135.28 sec. (168349.08 ticks)
+----------------------------------------------+------------------+-----------+
| Pr(Y = +1) = 1.0/(1.0 + exp(-(-2 + sco

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 12:15 PM | adding 169 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4538.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

...

 512099 33708        0.4564     3        0.4616        0.4564  1099083    1.15%           rho_6 U 512099 512039     33
 523561 33922    infeasible              0.4616        0.4565  1123628    1.13%          rho_11 D 523561 391791     31
 535532 34823        0.4587     1        0.4616        0.4565  1145301    1.12%           rho_4 D 535532 535531     42
 548507 36832        0.4565     3        0.4616        0.4565  1167807    1.12%           rho_3 U 548507 548506     36
 559734 36878        cutoff              0.4616        0.4566  1192079    1.10%           rho_6 U 559734 559733     45
 571489 36042        0.4574     1        0.4616        0.4566  1213560    1.09%           rho_2 D 571489 169178     33
 582505 34989        0.4577     1        0.4616        0.4567  1235693    1.07%           rho_3 D 582505 582503     41
 593449 33088        cutoff              0.4616        0.4567  1256940    1.07%           rho_6 U 593449 593448     45
 606497 33664        0.4582     1        0.4616 

10/20/19 @ 12:22 PM | running sequential rounding on 4 solutions
10/20/19 @ 12:22 PM | best objective value: 0.6931
10/20/19 @ 12:22 PM | sequential rounding produced 0 integer solutions
10/20/19 @ 12:22 PM | polishing 4 solutions
10/20/19 @ 12:22 PM | best objective value: 0.6931
10/20/19 @ 12:22 PM | polishing produced 4 integer solutions
10/20/19 @ 12:22 PM | initialization produced 7 feasible solutions
10/20/19 @ 12:22 PM | best objective value: 0.4569
10/20/19 @ 12:22 PM | ------------------------------------------------------------
10/20/19 @ 12:22 PM | completed initialization procedure
10/20/19 @ 12:22 PM | ------------------------------------------------------------
10/20/19 @ 12:22 PM | 193 rows in lookup table
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Parallel                                1
CPXPARAM_RandomSeed                              0
CPXPARAM_TimeLimit                               1000
CPXPARAM_MI

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 12:22 PM | adding 178 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4569.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 362490 55121        0.4402     8        0.4556        0.4402  1527107    3.38%           rho_6 U 362490 362487     34
 371158 56944        cutoff              0.4556        0.4402  1562257    3.38%           rho_7 N 371158 371156     35
 379296 59065        0.4427    11        0.4556        0.4402  1599489    3.38%           rho_5 U 379296 379295     30
 386899 60630        0.4550     9        0.4556        0.4402  1637655    3.38%           rho_7 U 386899 386898     28
 394038 61738        cutoff              0.4556        0.4402  1672500    3.38%           rho_8 U 394038 394037     35
 401121 63040        0.4554     7        0.4556        0.4402  1707867    3.38%           rho_7 N 401121 401099     31
 408581 63979        cutoff              0.4556        0.4402  1742450    3.38%          rho_14 D 408581 408580     43
 416419 65547        cutoff              0.4556        0.4402  1778922    3.38%           rho_8 U 416419 416417     24
 424810 67901        0.4472     5        0.4556 

 1409318 223251        0.4492     4        0.4556        0.4443  4742984    2.49%        alpha_10 D 1409318 1409316     35
 1446553 225143        0.4533     1        0.4556        0.4445  4836540    2.44%           rho_2 U 1446553 1211694     25
 1479352 223997        0.4517     9        0.4556        0.4447  4933167    2.39%          rho_13 U 1479352 935182     29
 1515290 226784        cutoff              0.4556        0.4450  5030441    2.34%          rho_14 U 1515290 1002202     33
 1550720 228202        0.4456     9        0.4556        0.4451  5130924    2.31%           rho_6 D 1550720 1192622     22
Elapsed time = 111.28 sec. (168097.56 ticks, tree = 58.72 MB, solutions = 3)
 1587100 231379        0.4463     7        0.4556        0.4452  5217055    2.29%           rho_6 D 1587100 1587099     42
 1626219 235511        0.4454     6        0.4556        0.4453  5298799    2.27%           rho_6 D 1626219 1626217     31
 1665032 239374        0.4456    11        0.4556        0.4454

In [26]:
riskslim_summary['KY_score'], np.mean(riskslim_summary['FL_train_score'])

(0.513644145212071, 0.5795062488806388)

In [28]:
#### save results
summary_felony2_fl_inter_model = {"CART": cart_summary,
                                    "EBM": ebm_summary,
                                    "Stumps": stumps_summary,
                                    "RiskSLIM": riskslim_summary}
%store summary_felony2_fl_inter_model

Stored 'summary_felony2_fl_inter_model' (dict)


In [29]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\FL Model\\interpretable\\Two Year\\"
results = [["Felony",             
            round(cart_summary['KY_score'],3), 
            round(ebm_summary['KY_score'], 3), 
            round(stumps_summary['KY_score'],3), 
            round(riskslim_summary['KY_score'], 3)]]
with open(path + 'Two Year FL_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)